In [ ]:
!nvidia-smi

# EDA

#### load data and chunk it

In [1]:
import os
import json
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import skimage.io as io
import matplotlib.patches as patches
import numpy as np
from numpy import unravel_index
import glob
import requests

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.5/dist-packages/pycocotools/coco.py:49: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.5/dist-packages/tornado/platfo

In [2]:
# look at data in air
jsonfiles = glob.glob("/root/data/gtsf_2.0/gtsf_air/labels/*.json")
data = []
for jf in jsonfiles: 
    data += json.load(open(jf))

In [4]:
# download images
for d in data:
    url = d['Labeled Data']
    path = "/root/data/gtsf_2.0/gtsf_air/images/{}".format(d['External ID'])
    if os.path.isfile(path):
        continue
    r = requests.get(url, allow_redirects=True)
    open(path, 'wb').write(r.content)

In [31]:
len([d for d in data if d["Label"] != "Skip"])

46

In [46]:
keypoints = ['eye', 
             'adipose fin', 
             'dorsal fin', 
             'pectoral fin', 
             'upper lip', 
             'anal fin']

In [47]:
good_data = []
for d in data:
    if d["Label"] != "Skip":
        cter = 0
        for k in keypoints:
            if k in d["Label"].keys():
                cter += 1
        if cter == len(keypoints):
            good_data.append(d)
print(len(good_data))

33


#### create generator

In [ ]:
import random

In [ ]:
def generator(data, batch_size, image_shape=[]):
    while True:
        xbatch = np.zeros((batch_size, tile, tile, 3), dtype=np.uint8)
        ybatch = np.zeros((batch_size, 2*len(keypoints)))
        for i in range(batch_size):
            
        yield (xbatch, ybatch)

In [ ]:
small_model = 

In [ ]:
json_format = json.load(open("./keypoints_181017010001_json_format.json"))
example = json_format[1]
image_path = "/root/data/small_pen_data_collection/181017010001/right_small-pen-test-site_1_1539774727592.jpg"
image = io.imread(image_path)

In [ ]:
classmap = {}
mapclass = {}
k = 1
for (i, c) in enumerate(example["Label"].keys()):
    if c == "Salmon":
        continue
    classmap[k] = c
    mapclass[c] = k
    k += 1

In [ ]:
def create_chunks(image, keypoints, tile):
    chunks = []
    height, width, _ = image.shape
    for i in range(height//tile):
        for j in range(width//tile):
            one_hot_vector = np.zeros((10)) # 10 classes = 9 kp + other
            tile_image = image[i*tile:i*tile+tile, j*tile:j*tile+tile]
            tile_keypoints = keypoints[i*tile:i*tile+tile, j*tile:j*tile+tile]
            # fish chunk
            if np.max(tile_keypoints) == 0:
                continue
                one_hot_vector[0] = 1
            else:
                one_hot_vector[int(np.max(tile_keypoints))] = 1
            chunks.append({"gt": one_hot_vector, 
                           "tile": tile_image, 
                           "coordinates": unravel_index(tile_keypoints.argmax(), tile_keypoints.shape)})
                
    return chunks

In [ ]:
# chunks = create_chunks(image, keypoints, 224)

In [ ]:
# len(chunks)

In [ ]:
# for z in range(9):
#     plt.imshow(chunks[z]["tile"])
#     print(chunks[z]["gt"])
#     print(classmap[np.argmax(chunks[z]["gt"])])
#     coord = chunks[z]["coordinates"]
#     plt.scatter(coord[1], coord[0])
#     plt.show()

### create generator

In [ ]:
import random

In [ ]:
def generator(json_format, batch_size, tile):
    while True:
        xbatch = np.zeros((batch_size, tile, tile, 3), dtype=np.uint8)
        y1 = np.zeros((batch_size, 2))
        y2 = np.zeros((batch_size, 10))
        ct = 0
        while ct < batch_size:
            random_sample = np.random.choice(json_format)
#              print(random_sample['Labeled Data'])
            random_image_path = os.path.join("/root/data/small_pen_data_collection/", "/".join(random_sample['Labeled Data'].split("/")[-2:]))
            image = io.imread(random_image_path)
            
            # create keypoints map
            k = 1
            keypoints = np.zeros((image.shape[0], image.shape[1]))
#             plt.imshow(image)
            for (i, c) in enumerate(random_sample["Label"].keys()):
                if c == "Salmon":
                    continue
                y, x = random_sample["Label"][c][0]["geometry"]["x"], random_sample["Label"][c][0]["geometry"]["y"]
                keypoints[x, y] = mapclass[c]
#                 plt.scatter(y,x,color="r")
#             plt.show()

            
            chunks = create_chunks(image, keypoints, tile)
            random.shuffle(chunks)
            randlen = np.random.randint(low=1, high=len(chunks)+1)
            for c in chunks[:randlen]:
                if ct == batch_size:
                    continue
                xbatch[ct, ...] = c["tile"]
                y1[ct, ...] = c["coordinates"]
                y2[ct, ...] = c["gt"]
                ct += 1
        yield (xbatch, {'reg': y1, 'classif': y2})

In [ ]:
train_gen = generator(json_format, 8, 224)

In [ ]:
xb, yb = next(train_gen)

In [ ]:
yb

In [ ]:
for z in range(8):
    plt.imshow(xb[z, ...])
    plt.scatter(yb["reg"][z][1], yb["reg"][z][0])
#     reg, classif = model.predict_on_batch(np.expand_dims(chunks[z]["tile"], axis=0))
    # classif = model.predict_on_batch(np.expand_dims(chunks[z]["tile"], axis=0))
#     print(chunks[z]["gt"])
#     print(np.round(classif[0]*1000)/1000.0)
#     print(classmap[np.argmax(classif[0])])
#     print(reg[0])
#     coord = chunks[z]["coordinates"]
#     reg = reg[0]
#     plt.scatter(coord[1], coord[0], color="r")
#     plt.scatter(reg[1], reg[0], color="b")
    plt.show()

# Create model

In [ ]:
!nvidia-smi

In [ ]:
import os
from keras.applications.mobilenet import MobileNet
from keras.applications.resnet50 import ResNet50
from keras import layers
from keras.models import Model
from utils import depthwise_conv_block, conv_block

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
tile = 224

In [ ]:
alpha = 1.0
dropout=1e-3
classes = 2
depth_multiplier = 1
shape = (1, 1, int(1024 * alpha))

In [ ]:
img_input = layers.Input(shape=[tile, tile, 3])
x = conv_block(img_input, 32, alpha, strides=(2, 2))
x = depthwise_conv_block(x, 64, alpha, depth_multiplier, block_id=1)

x = depthwise_conv_block(x, 128, alpha, depth_multiplier,
                          strides=(2, 2), block_id=2)
x = depthwise_conv_block(x, 128, alpha, depth_multiplier, block_id=3)

x = depthwise_conv_block(x, 256, alpha, depth_multiplier,
                          strides=(2, 2), block_id=4)
x = depthwise_conv_block(x, 256, alpha, depth_multiplier, block_id=5)

x = depthwise_conv_block(x, 512, alpha, depth_multiplier,
                          strides=(2, 2), block_id=6)

# head1 
h1 = depthwise_conv_block(x, 512, alpha, depth_multiplier, block_id=7)
h1 = depthwise_conv_block(h1, 512, alpha, depth_multiplier, block_id=8)
h1 = depthwise_conv_block(h1, 512, alpha, depth_multiplier, block_id=9)
h1 = depthwise_conv_block(h1, 512, alpha, depth_multiplier, block_id=10)
h1 = depthwise_conv_block(h1, 512, alpha, depth_multiplier, block_id=11)

h1 = depthwise_conv_block(h1, 1024, alpha, depth_multiplier, strides=(2, 2), block_id=12)
h1 = depthwise_conv_block(h1, 1024, alpha, depth_multiplier, block_id=13)
    
h1 = layers.GlobalAveragePooling2D()(h1)
h1 = layers.Reshape(shape, name='reshape_1')(h1)
h1 = layers.Dropout(dropout, name='dropout1')(h1)
h1 = layers.Conv2D(2, (1, 1), padding='same', name='conv_preds1')(h1)
h1 = layers.Activation('linear', name='linear')(h1)
reg = layers.Reshape((2,), name='reg')(h1)

# head2
h2 = depthwise_conv_block(x, 512, alpha, depth_multiplier, block_id=14)
h2 = depthwise_conv_block(h2, 512, alpha, depth_multiplier, block_id=15)
h2 = depthwise_conv_block(h2, 512, alpha, depth_multiplier, block_id=16)
h2 = depthwise_conv_block(h2, 512, alpha, depth_multiplier, block_id=17)
h2 = depthwise_conv_block(h2, 512, alpha, depth_multiplier, block_id=18)

h2 = depthwise_conv_block(h2, 1024, alpha, depth_multiplier, strides=(2, 2), block_id=19)
h2 = depthwise_conv_block(h2, 1024, alpha, depth_multiplier, block_id=20)

h2 = layers.GlobalAveragePooling2D()(h2)
h2 = layers.Reshape(shape, name='reshape_2')(h2)
h2 = layers.Dropout(dropout, name='dropout2')(h2)
h2 = layers.Conv2D(10, (1, 1), padding='same', name='conv_preds2')(h2)
h2 = layers.Activation('softmax', name='act_softmax')(h2)
classif = layers.Reshape((10,), name='classif')(h2)


model = Model(inputs=[img_input], outputs=[reg, classif])

In [ ]:
model.compile(optimizer='adam', loss={'reg': 'mean_squared_error', 'classif': 'categorical_crossentropy'}) #, loss_weights={'reg': 0.5, 'classif': 0.5})

# Training

In [ ]:
history = model.fit_generator(train_gen,
                    steps_per_epoch= len(json_format) // 8, 
                    epochs=500)

In [ ]:
h = history.history

In [ ]:
with open("./history.json", "w") as f:
    json.dump(h, f)

In [ ]:
plt.plot(h["reg_loss"])
plt.plot(h["classif_loss"])
plt.ylim([0, 500])
plt.show()

In [ ]:
model.save("/root/data/models/biomass/key_points_detection.h5")

## Testing

In [ ]:
for z in range(9):
    plt.imshow(chunks[z]["tile"])
    reg, classif = model.predict_on_batch(np.expand_dims(chunks[z]["tile"], axis=0))
    # classif = model.predict_on_batch(np.expand_dims(chunks[z]["tile"], axis=0))
    print(chunks[z]["gt"])
    print(np.round(classif[0]*1000)/1000.0)
    print(classmap[np.argmax(classif[0])])
    print(reg[0])
    coord = chunks[z]["coordinates"]
    reg = reg[0]
    plt.scatter(coord[1], coord[0], color="r")
    plt.scatter(reg[1], reg[0], color="b")
    plt.show()

In [ ]:
preds

In [ ]:
body_polygon = example["Label"]["Salmon"][0]["geometry"]
coordinates = np.array([[k["x"], k["y"]] for k in body_polygon])
y1, y2 = np.min(coordinates[:, 0]), np.max(coordinates[:, 0])
x1, x2 = np.min(coordinates[:, 1]), np.max(coordinates[:, 1])
print(x1, x2, y1, y2)

In [ ]:
f, ax = plt.subplots(1, figsize=(15, 20))
ax.imshow(image[x1:x2, y1:y2])
# plt.axis("off")
plt.show()

In [ ]:
height = x2-x1
width = y2-y1
tile = 512

In [ ]:
crop = image[x1:x2, y1:y2]

In [ ]:
flatten_crop = list(crop.flatten())

In [ ]:
flatten_chunks = [flatten_crop[i:i + 3*tile**2] for i in range(0, len(flatten_crop), 3*tile**2)]

In [ ]:
import json

In [ ]:
with open("./history.json", "w") as f: 
    json.dump()

In [ ]:
with open("./history.json", "w") as f: 

### create generator